In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Configs

In [4]:
LABEL2IDX = {
    'O': 0,
    'B-DAT': 1,
    'B-PER': 2,
    'B-ORG': 3,
    'B-LOC': 4,
    'B-EVE': 5,
    'I-DAT': 6,
    'I-PER': 7,
    'I-ORG': 8,
    'I-LOC': 9,
    'I-EVE': 10
}

IDX2LABEL = {i: k for k, i in LABEL2IDX.items()}

CLS = [101]
SEP = [102]
VALUE_TOKEN = [0]
MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
EPOCHS = 4
NUM_CLASS = 11
LEARNING_RATE = 5e-5

In [5]:
from huggingface_hub import notebook_login

notebook_login()


In [6]:
from datasets import concatenate_datasets, load_dataset
from transformers import AutoTokenizer
import ast
model_name = 'sbunlp/fabert'
# model_name = 'PartAI/TookaBERT-Base'
# model_name = 'PartAI/TookaBERT-Large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_files = "/content/drive/MyDrive/Colab Notebooks/NER-datasets/shuffled-100000-2.csv"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/552k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
def parse_tokens_and_labels(example):
    # Ensure 'tokens' and 'labels' are in the expected format
    if isinstance(example['tokens'], str):
        example['tokens'] = ast.literal_eval(example['tokens'])
    if isinstance(example['labels'], str):
        example['labels'] = ast.literal_eval(example['labels'])
    return example


In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],truncation=True,max_length=512,
 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [12]:
dataset = load_dataset('csv', data_files=data_files)
dataset = dataset['train'].select(range(60000))
# Apply the parsing function to the entire dataset
parsed_dataset = dataset.map(parse_tokens_and_labels)
# rename labels to net_tags
parsed_dataset = parsed_dataset.rename_column("labels", "ner_tags")
tokenized_dataset = parsed_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=parsed_dataset.column_names,)

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [14]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 60000
})

In [15]:
# Printing the values and lengths of each column in the first item of tokenized_dataset
print(f"Input IDs: {tokenized_dataset[0]['input_ids']}, length: {len(tokenized_dataset[0]['input_ids'])}")
print(f"Attention Mask: {tokenized_dataset[0]['attention_mask']}, length: {len(tokenized_dataset[0]['attention_mask'])}")
print(f"Labels: {tokenized_dataset[0]['labels']}, length: {len(tokenized_dataset[0]['labels'])}")

Input IDs: [101, 5055, 4588, 43143, 2303, 2588, 3087, 2342, 2307, 3570, 49470, 2697, 2297, 5042, 34007, 15941, 6663, 2342, 622, 2297, 5042, 2588, 17710, 48332, 2307, 43143, 8080, 2303, 2402, 2352, 3775, 3249, 4588, 2549, 622, 2491, 2303, 11707, 2307, 2902, 2601, 3610, 2303, 2402, 2445, 2297, 9477, 2293, 3793, 791, 2003, 2299, 3656, 34007, 15941, 2856, 622, 2314, 3249, 2297, 3845, 3845, 14467, 2342, 117, 102], length: 66
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], length: 66
Labels: [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 4, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, -100, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], length: 66


In [16]:
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][1]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][-2]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][-1]))

[CLS]
نحوه
.
[SEP]


In [17]:
print(tokenizer.convert_ids_to_tokens(tokenized_dataset[0]['input_ids']))


['[CLS]', 'نحوه', 'دعوت', 'پیرنیا', 'از', 'وی', 'چنین', 'بود', 'که', 'روزی', 'نجا', '##حی', 'در', 'منزل', 'عبدالعلی', 'وزیری', 'مهمان', 'بود', 'و', 'در', 'منزل', 'وی', 'سنتور', 'میڸزده', 'که', 'پیرنیا', 'بلافاصله', 'از', 'او', 'برای', 'اجرای', 'برنامه', 'دعوت', 'میڸکند', 'و', 'پس', 'از', 'امتحانی', 'که', 'توسط', 'چند', 'استاد', 'از', 'او', 'میڸشود', 'در', 'گله', '##ای', 'شماره', '۲', '##۰', 'به', 'اتفاق', 'عبدالعلی', 'وزیری', 'شرکت', 'و', 'این', 'برنامه', 'در', 'دستگاه', 'دستگاه', 'همایون', 'بود', '.', '[SEP]']


In [18]:
# Split the combined dataset into train, validation, and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_eval_split = train_test_split['test'].train_test_split(test_size=0.5)

train_dataset = train_test_split['train']
eval_dataset = train_eval_split['train']
test_dataset = train_eval_split['test']
# Print out info of each dataset
print(f"Training set: {train_dataset} samples")
print(f"Validation set: {eval_dataset} samples")
print(f"Test set: {test_dataset} samples")


Training set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 48000
}) samples
Validation set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 6000
}) samples
Test set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 6000
}) samples


In [19]:
from transformers import DataCollatorForTokenClassification
import torch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")
test_dataset.set_format("torch")

In [21]:
train_sample = train_dataset[0]
print(type(train_sample['input_ids']))

<class 'torch.Tensor'>


In [22]:
# check the paddings with -100s
batch = data_collator([train_dataset[i] for i in range(2)])
batch["labels"]

tensor([[-100,    0, -100, -100,    1,    6,    6, -100, -100, -100,    0, -100,
         -100,    0, -100, -100,    0,    0,    0,    0,    0, -100, -100,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0, -100],
        [-100,    0, -100,    1,    6, -100,    6,    6,    6, -100, -100, -100,
            0,    4, -100,    9,    0,    0, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100]])

In [23]:
for i in range(2):
    print(train_dataset[i]["labels"])

tensor([-100,    0, -100, -100,    1,    6,    6, -100, -100, -100,    0, -100,
        -100,    0, -100, -100,    0,    0,    0,    0,    0, -100, -100,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, -100])
tensor([-100,    0, -100,    1,    6, -100,    6,    6,    6, -100, -100, -100,
           0,    4, -100,    9,    0,    0, -100])


In [ ]:
IDX2LABEL

{0: 'O',
 1: 'B-DAT',
 2: 'B-PER',
 3: 'B-ORG',
 4: 'B-LOC',
 5: 'B-EVE',
 6: 'I-DAT',
 7: 'I-PER',
 8: 'I-ORG',
 9: 'I-LOC',
 10: 'I-EVE'}

In [ ]:
LABEL2IDX

{'O': 0,
 'B-DAT': 1,
 'B-PER': 2,
 'B-ORG': 3,
 'B-LOC': 4,
 'B-EVE': 5,
 'I-DAT': 6,
 'I-PER': 7,
 'I-ORG': 8,
 'I-LOC': 9,
 'I-EVE': 10}

In [24]:
!pip install seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=6938fdccf1a83aa8f64e19e135d5536b5d1cf7460363720e1282557483adb870
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [25]:
label_list = [k for k,v in LABEL2IDX.items()]
label_list

['O',
 'B-DAT',
 'B-PER',
 'B-ORG',
 'B-LOC',
 'B-EVE',
 'I-DAT',
 'I-PER',
 'I-ORG',
 'I-LOC',
 'I-EVE']

In [26]:
import numpy as np
from seqeval.metrics import classification_report
import evaluate

accuracy = evaluate.load('accuracy')
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #############
    cr = classification_report(true_labels, true_predictions,digits=4)
    fname = "report.txt"
    file = open(fname,'w')
    file.write(cr)
    file.close()
    #############
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [27]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoConfig
model_name = 'pouria82/fabert-base_finetuned'

config = AutoConfig.from_pretrained(model_name)
# Modify the dropout rates
config.attention_probs_dropout_prob = 0.2  #  default (0.1)
config.hidden_dropout_prob = 0.2  # default (0.1)

model = AutoModelForTokenClassification.from_pretrained(model_name, config=config)

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

In [28]:
samples = train_dataset[:8]
batch = data_collator([train_dataset[i] for i in range(i)])
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([1, 53]),
 'token_type_ids': torch.Size([1, 53]),
 'attention_mask': torch.Size([1, 53]),
 'labels': torch.Size([1, 53])}

In [29]:
 model.config.max_position_embeddings

512

In [30]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 48000
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 6000
})


In [ ]:
from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    "faBERT-finetuned-2448",
    learning_rate= LEARNING_RATE,
    per_device_train_batch_size= TRAIN_BATCH_SIZE,
    per_device_eval_batch_size= EVAL_BATCH_SIZE,
    num_train_epochs= 10,
    weight_decay= 0.01,#*********************
    warmup_ratio = 0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    lr_scheduler_type="linear",
    greater_is_better=True,
    push_to_hub=True,
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.176700,0.171598,0.641628,0.679677,0.660105,0.939140
2,0.148000,0.189052,0.630087,0.723382,0.673519,0.937482
3,0.112900,0.196787,0.666047,0.675081,0.670534,0.942390


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.176700,0.171598,0.641628,0.679677,0.660105,0.939140
2,0.148000,0.189052,0.630087,0.723382,0.673519,0.937482
3,0.112900,0.196787,0.666047,0.675081,0.670534,0.942390
4,0.084800,0.214892,0.667053,0.693090,0.679823,0.942240
5,0.058300,0.267811,0.644019,0.686384,0.664527,0.939585
6,0.041700,0.298820,0.653932,0.706804,0.679341,0.940811
7,0.028100,0.338871,0.667251,0.688418,0.677669,0.941335


In [35]:
model.config.num_labels

11

In [36]:
validation_results = trainer.evaluate()

In [37]:
validation_results

{'eval_loss': 0.21489229798316956,
 'eval_precision': 0.6670534484008993,
 'eval_recall': 0.6930901966694296,
 'eval_f1': 0.6798226164079822,
 'eval_accuracy': 0.94223975776483,
 'eval_runtime': 58.1786,
 'eval_samples_per_second': 103.131,
 'eval_steps_per_second': 12.891,
 'epoch': 7.0}

In [38]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
        )
test_results = trainer.evaluate()

In [41]:
test_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 6000
})

In [39]:
test_results

{'eval_loss': 0.217198446393013,
 'eval_precision': 0.6515628431300783,
 'eval_recall': 0.6902140198511166,
 'eval_f1': 0.6703317392777799,
 'eval_accuracy': 0.9411168962696548,
 'eval_runtime': 48.2895,
 'eval_samples_per_second': 124.251,
 'eval_steps_per_second': 15.531}

In [42]:
with open('report.txt') as f:
    for line in f.readlines():
        print(line)

              precision    recall  f1-score   support



         DAT     0.7295    0.7873    0.7573      4490

         EVE     0.5020    0.6054    0.5489       408

         LOC     0.5993    0.6190    0.6090      3861

         ORG     0.5853    0.6328    0.6081      1811

         PER     0.6660    0.6806    0.6732      2326



   micro avg     0.6516    0.6902    0.6703     12896

   macro avg     0.6164    0.6650    0.6393     12896

weighted avg     0.6516    0.6902    0.6702     12896



In [43]:
model.push_to_hub("faBERT-base_finetuned-2448")

model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pouria82/faBERT-base_finetuned-2448/commit/bee3dd012bf95255d49dc452649eb129484e7af3', commit_message='Upload BertForTokenClassification', commit_description='', oid='bee3dd012bf95255d49dc452649eb129484e7af3', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
from transformers import AutoConfig
print(AutoConfig.from_pretrained("pouria82/faBERT-base_finetuned-2448"))

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

BertConfig {
  "_name_or_path": "pouria82/faBERT-base_finetuned-2448",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.2,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-DAT",
    "2": "B-PER",
    "3": "B-ORG",
    "4": "B-LOC",
    "5": "B-EVE",
    "6": "I-DAT",
    "7": "I-PER",
    "8": "I-ORG",
    "9": "I-LOC",
    "10": "I-EVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DAT": 1,
    "B-EVE": 5,
    "B-LOC": 4,
    "B-ORG": 3,
    "B-PER": 2,
    "I-DAT": 6,
    "I-EVE": 10,
    "I-LOC": 9,
    "I-ORG": 8,
    "I-PER": 7,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4